In [30]:
# auto reload libraries (you do need to re-import libraries if you make changes)
%load_ext autoreload
%autoreload 2

# base 
import pandas as pd
import numpy as np
from pprint import pprint
import os

# preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer,KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import TruncatedSVD
# variable importance
from sklearn.inspection import permutation_importance

# visualize
import matplotlib.pyplot as plt

# models
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor


# own defined functions/classes 
from preprocessing.preprocess_pipe import MakeLowerCase
from preprocessing.preprocess_pipe import HdResolutionCategorizer
from preprocessing.preprocess_pipe import StorageCategorizer
from preprocessing.preprocess_pipe import SsdCategorizer

from preprocessing.preprocess_pipe import print_missing
from preprocessing.preprocess_pipe import calculate_perf
from preprocessing.preprocess_pipe import custom_scoring_func_single_p

from modelling.weight_samples import weights_samples

from postprocessing.postprocessing import plot_predictions_results
from postprocessing.postprocessing import plot_residuals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# global parameters
pd.set_option('display.max_columns', 500)
# style for plotting
plt.style.use('ggplot')
# interactive plotting
%matplotlib widget
# run grid search
RUN_GRID_SEARCH = True
# set working directory
uppath = lambda _path, n: os.sep.join(_path.split(os.sep)[:-n])
__file__ = 'C:\\Users\\Pieter-Jan\\Documents\\KuLeuven\\Semester2\\AA\\AAForB_Assignment2\\code\\PieterJan'
out = uppath(__file__, 2)
os.chdir(out)
print(os.getcwd())

C:\Users\Pieter-Jan\Documents\KuLeuven\Semester2\AA\AAForB_Assignment2


# Step 1: import data and split in train and validation set
The validation set is more our own kind of test set

In [3]:
# read in trainig and validation data
# use the same data split as we did in R
df_all_train = pd.read_csv("data\\train.csv", sep=',')
df_test = pd.read_csv("data\\test.csv", sep=',')

print(f'Dimensions of all training data {df_all_train.shape}')
print(f'Dimension test data {df_test.shape}')

Dimensions of all training data (510, 22)
Dimension test data (222, 20)


In [4]:
# split in training and validation set
df_train = df_all_train.sample(frac=0.75, random_state=0, replace=False)
df_val = df_all_train.drop(df_train.index)

# reset index, if you don't resit missing rows get inserted in the pipeline
# see: https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
df_train = df_train.reset_index().drop('index',axis=1)
df_val = df_val.reset_index().drop('index',axis=1)

# Step 2: define pipeline

In [5]:
#Numerical features to pass down the numerical pipeline 
numerical_features = ['screen_size' ,'pixels_x','pixels_y',
                      'ram', 'weight','ssd','storage']

#Categrical features to pass down the categorical pipeline 
categorical_features = ['brand','base_name', 'screen_surface','touchscreen',
                        'cpu','cpu_details','detachable_keyboard',
                        'discrete_gpu','gpu', 'os','os_details']

# define all unique features
features = np.unique(numerical_features).tolist() + np.unique(categorical_features).tolist()

# target variables
target = ['min_price','max_price']

In [6]:
# train
X_train = df_train[features]
y_train = df_train[target]

# validation (this is kind our own test set)
X_val  = df_val[features]
y_val = df_val[target]

# train_validation (this is all training data we have) for fitting the model
X_all_train = df_all_train[features]
y_all_train = df_all_train[target]

# test
X_test = df_test[features]

In [7]:
# you can add many more and 
# you can even define custom preprocessing steps like 'MakeLowerCase()'

# pipeline  uses only numerical features,
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    #('imputer', KNNImputer(n_neighbors=5, weights='uniform',metric='nan_euclidean')),
    ('imputer', IterativeImputer(missing_values=np.nan, random_state=0, n_nearest_features=None, sample_posterior=True)),
    #('transformation', PowerTransformer(method='yeo-johnson',standardize=False)),
     #Scale features using statistics that are robust to outliers.
    ('scaler', RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0)))]) 

# pipeline use only categorical features
categorical_transformer = Pipeline(steps=[ 
    ('lowercase', MakeLowerCase()), # lower cases all columns containing strings
    #('sd_category' ,SsdCategorizer(drop_original_feature=True)),
    #('storage_category', StorageCategorizer(drop_original_feature=True)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# add both preprocessing pipelines in one pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [8]:
# this is how the preporcessed pipeline looks like (just to have an idea)
pd.DataFrame(preprocessor.fit_transform(X_train)).shape

(382, 1)

# Step 3: add models to pipeline


### Stacking

- support vector regression
- random forest

In [41]:
# train your final model on all data with best parameters 

# support vector machine
svr = SVR(
    tol = 0.0001,
    shrinking = True,
    kernel = 'rbf',
    gamma = 'scale',
    epsilon = 0.001,
    degree = 7,
    coef0 = 0.1,
    C = 1
)

# random forest
rf = RandomForestRegressor(
     criterion='mse',
     n_estimators=2710, 
     max_depth=105,
     max_features='log2',
     min_samples_split=0.001,
     min_samples_leaf=0.001,
     bootstrap=False,
     n_jobs=-1,
     random_state=0
)

xgb = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=736,
    max_depth=7,
    learning_rate=0.01,
    random_state=0
)

# stack
estimators = [
    ('Random Forest', rf),
    ('SVR', svr),
    ('Xgb',xgb)
]
stacking_regressor = StackingRegressor(
    estimators=estimators, final_estimator=svr
)


# add to pipeline
# 1) min price
pipeline_min_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', stacking_regressor)],
              verbose=True)

# 2) min price
pipeline_max_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', stacking_regressor)],
              verbose=True)

transformer_target = PowerTransformer(method='yeo-johnson',standardize=False)
scale_target = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0))
pipeline_y = Pipeline(memory=None,
              steps=[('transformer', transformer_target)])

# 1) min price
pipeline_min_p_update = TransformedTargetRegressor(regressor=pipeline_min_p, 
                                         transformer=pipeline_y)
# 2) max price
pipeline_max_p_update = TransformedTargetRegressor(regressor=pipeline_max_p, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand

# 1) min price
weights_train_min_p = weights_samples(y_train.iloc[:,0], order=0)
pipeline_min_p = pipeline_min_p_update.fit(X_train, y_train.iloc[:,0])

# 2) max price
weights_train_max_p = weights_samples(y_train.iloc[:,1], order=0)
pipeline_max_p = pipeline_max_p_update.fit(X_train, y_train.iloc[:,1])

Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.5s
[Pipeline] ......... (step 2 of 2) Processing regressor, total= 1.7min
Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.5s
[Pipeline] ......... (step 2 of 2) Processing regressor, total= 2.1min


In [42]:
# performance on data where the model was fit one (should be very low)
pred_train_min_p = pipeline_min_p.predict(X_train)
pred_train_max_p = pipeline_max_p.predict(X_train)

# calculate performance
pred_train = pd.DataFrame([pred_train_min_p,pred_train_max_p]).T
calculate_perf(y_train, pred_train)

{'minimum price': 61.107267110285655,
 'maximum price': 52.705782185847305,
 'total error': 113.81304929613296}

##### Performance on validation data

In [43]:
# performance on validation data
pred_val_min_p = pipeline_min_p.predict(X_val)
pred_val_max_p = pipeline_max_p.predict(X_val)

# calculate performance 
pred_val = pd.DataFrame([pred_val_min_p,pred_val_max_p]).T
calculate_perf(y_val, pred_val)

{'minimum price': 113.51223644477835,
 'maximum price': 118.985026816365,
 'total error': 232.49726326114336}

In [44]:
# save predictions on validation data
# submission format
submission_format_validation = pd.DataFrame.from_dict(
 {'ID':df_val['id'].values,
 'MIN':pred_val_min_p,
 'MAX':pred_val_max_p}).set_index('ID')

# write to csv
submission_format_validation.to_csv('code\\PieterJan\\python\\output\\validation\\stacking.csv' ,
                            header=True, index=True, sep=',')

### C) Post processing

In [45]:
fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)

# fitted against true predictions minimum price
plot_predictions_results(ax=axs[0], 
                        y_true=y_val.iloc[:,0], 
                        y_pred=pred_val_min_p, 
                        title="SVM Min. Price Test Set", 
                        log_scale=True)

# fitted against true predictions maximum price
plot_predictions_results(ax=axs[1], 
                        y_true=y_val.iloc[:,1], 
                        y_pred=pred_val_max_p, 
                        title="SVM Max. Price Test Set", 
                        log_scale=True)

# residuals minimum price
plot_residuals(ax=axs[2], 
               y_true=y_val.iloc[:,0], 
               y_pred=pred_val_min_p, 
               title="", 
               log_scale=False,
               order=1)

# residuals maximum price
plot_residuals(ax=axs[3], 
               y_true=y_val.iloc[:,1], 
               y_pred=pred_val_max_p, 
               title="", 
               log_scale=False,
               order=1)
fig.tight_layout()
plt.savefig('code\\PieterJan\\python\\output\\figures\\stacking\\fig1.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
plt.figure(figsize=(8,4))
plt.plot(pred_val_max_p - pred_val_min_p, label="Predictions Test", linestyle="--")
plt.plot(y_val.iloc[:,1] - y_val.iloc[:,0], label="Truth Test", linestyle=':')
plt.xlabel("Observations")
plt.ylabel("Max. Price - Min. Price")
plt.legend(loc="upper right");
plt.savefig('code\\PieterJan\\python\\output\\figures\\stacking\\fig2.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
vip_val_min_p = permutation_importance(pipeline_min_p_update, X_val, y_val.iloc[:,0], n_repeats=10,
                                random_state=1, n_jobs=3)

vip_val_max_p = permutation_importance(pipeline_min_p_update, X_val, y_val.iloc[:,1], n_repeats=10,
                                random_state=1, n_jobs=3)

sorted_idx_min_p = vip_val_min_p.importances_mean.argsort()
sorted_idx_max_p = vip_val_max_p.importances_mean.argsort()

C:\Users\Pieter-Jan\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\Pieter-Jan\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [27]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
ax = np.ravel(ax)
# minimum price
ax[0].boxplot(vip_val_min_p.importances[sorted_idx_min_p].T,
           vert=False, labels=X_val.columns[sorted_idx_min_p])
ax[0].set_title("Permutation Importances: Min. Price (Test set)", fontsize=9)
ax[0].xaxis.set_tick_params(labelsize=8)
ax[0].yaxis.set_tick_params(labelsize=8)

# maximum price
ax[1].boxplot(vip_val_min_p.importances[sorted_idx_max_p].T,
           vert=False, labels=X_val.columns[sorted_idx_max_p])
ax[1].set_title("Permutation Importances: Max. Price (Test set)", fontsize=9)
ax[1].xaxis.set_tick_params(labelsize=8)
ax[1].yaxis.set_tick_params(labelsize=8)
fig.tight_layout()
plt.show()
plt.savefig('code\\PieterJan\\python\\output\\figures\\stacking\\fig3.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### D) Predictions test data

In [47]:
# train your final model on all data with best parameters 

# support vector machine
svr = SVR(
    tol = 0.0001,
    shrinking = True,
    kernel = 'rbf',
    gamma = 'scale',
    epsilon = 0.001,
    degree = 7,
    coef0 = 0.1,
    C = 1
)

# random forest
rf = RandomForestRegressor(
     criterion='mse',
     n_estimators=2710, 
     max_depth=105,
     max_features='log2',
     min_samples_split=0.001,
     min_samples_leaf=0.001,
     bootstrap=False,
     n_jobs=-1,
     random_state=0
)

xgb = XGBRegressor(
    objective = 'reg:squarederror',
    n_estimators = 736,
    max_depth = 7,
    learning_rate = 0.01
)

# stack
estimators = [
    ('Random Forest', rf),
    ('SVR', svr),
    ('Xgb',xgb)
]
stacking_regressor = StackingRegressor(
    estimators=estimators, final_estimator=svr
)



# add to pipeline

# 1) min price
pipeline_min_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', stacking_regressor)],
              verbose=True)

# 2) min price
pipeline_max_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', stacking_regressor)],
              verbose=True)

transformer_target = PowerTransformer(method='yeo-johnson',standardize=False)
scale_target = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0))
pipeline_y = Pipeline(memory=None,
              steps=[('transformer', transformer_target)])

# 1) min price
pipeline_min_p_final = TransformedTargetRegressor(regressor=pipeline_min_p, 
                                         transformer=pipeline_y)
# 2) max price
pipeline_max_p_final = TransformedTargetRegressor(regressor=pipeline_max_p, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand

# 1) min price
weights_train_min_p = weights_samples(y_train.iloc[:,0], order=0)
pipeline_min_p_final.fit(X_train, y_train.iloc[:,0])

# 2) max price
weights_train_max_p = weights_samples(y_train.iloc[:,1], order=0)
pipeline_max_p_final.fit(X_train, y_train.iloc[:,1])

Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.4s
[Pipeline] ......... (step 2 of 2) Processing regressor, total= 1.9min
Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.5s
[Pipeline] ......... (step 2 of 2) Processing regressor, total= 2.1min


TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=Pipeline(memory=None,
                                              steps=[('preprocessor',
                                                      ColumnTransformer(n_jobs=None,
                                                                        remainder='drop',
                                                                        sparse_threshold=0.3,
                                                                        transformer_weights=None,
                                                                        transformers=[('num',
                                                                                       Pipeline(memory=None,
                                                                                                steps=[('imputer',
                                                                                                        Itera

In [48]:
# performance on all data where the model was fit one (should be very low)
pred_all_train_min_p = pipeline_min_p_final.predict(X_all_train)
pred_all_train_max_p = pipeline_max_p_final.predict(X_all_train)

# calculate performance
pred_all_train = pd.DataFrame([pred_all_train_min_p, pred_all_train_max_p]).T
calculate_perf(y_all_train, pred_all_train)

{'minimum price': 74.34742003983828,
 'maximum price': 69.33214564651429,
 'total error': 143.6795656863526}

In [49]:
# make predictions on test data
pred_test_min_p = pipeline_min_p_final.predict(X_test)
pred_test_max_p = pipeline_max_p_final.predict(X_test)

# submission format
submission_format = pd.DataFrame.from_dict(
 {'ID':df_test['id'].values,
 'MIN':pred_test_min_p,
 'MAX':pred_test_max_p}).set_index('ID')

# write to csv
submission_format.to_csv('code\\PieterJan\\python\\output\\submission\\stacking_python.csv' ,
                            header=True, index=True, sep=',')